<font size="5" color="yellow">
Inverse Kinematics
</font>



<font size="2.5">
<p>The inverse kinematics (IK) represents calculation of the configuration of the robot given the coordinates of the tool frame (its position and
orientation).
</p>
<ol>
<li>Solve forward kinematics</li>
<li>Calculate the coordinates of the intersection between the rotation axes given the coordinated of the tool</li>
<li>Solve position IK and get $\theta_1$ , $\theta_2$ and $\theta_3$</li>
<li>Calculate $R^{0}_{3}$ from forward kinematics</li>
<li>Calculate matrix $R^{3}_{6}$</li>
</ol>
</font>

In [1]:
import numpy as np
import math

In [2]:
a2 = 1
d1, d4, d6 = 1, 1, 1
theta = [np.pi/3, np.pi/2, np.pi/4, np.pi/3, 0, np.pi]
params = np.array([[0, a2, 0, 0, 0, 0], 
                  [np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0], 
                  [d1, 0, 0, d4, 0, d6], 
                  [theta[0], theta[1], theta[2] + np.pi/2, theta[3], theta[4], theta[5]]])

In [3]:
params

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.57079633,  0.        ,  1.57079633, -1.57079633,  1.57079633,
         0.        ],
       [ 1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.        ],
       [ 1.04719755,  1.57079633,  2.35619449,  1.04719755,  0.        ,
         3.14159265]])

In [4]:
def homo_trans_mat(a:float, alpha:float, d:float, theta:float) -> np.ndarray:
    '''compute homogeneous transformation matrix of ith link w.r.t. (i-1)th link based on DH parameters table
    '''
    c_theta = 0 if (theta == np.pi/2 or theta == -np.pi/2) else np.cos(theta)
    c_alpha = 0 if (alpha == np.pi/2 or alpha == -np.pi/2) else np.cos(alpha)
    
    return np.array([[c_theta, -np.sin(theta)*c_alpha, np.sin(theta)*np.sin(alpha), a*c_theta], 
                     [np.sin(theta), c_theta*c_alpha, -c_theta*np.sin(alpha), a*np.sin(theta)], 
                     [0, np.sin(alpha), c_alpha, d], 
                     [0, 0, 0, 1]])

In [5]:
homo_trans_mat(params[:, 0][0], params[:, 0][1], params[:, 0][2], params[:, 0][3])

array([[ 0.5      , -0.       ,  0.8660254,  0.       ],
       [ 0.8660254,  0.       , -0.5      ,  0.       ],
       [ 0.       ,  1.       ,  0.       ,  1.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [6]:
def cal_homo_trans_mat_0_to_n(n:int) -> np.ndarray:
    '''calculate homogeneous transformation matrix of nth frame w.r.t. base(0th frame)
    '''
    T = np.eye(4)
    for i in range(n):
        T = T @ homo_trans_mat(params[:, i][0], params[:, i][1], params[:, i][2], params[:, i][3])
    return T

In [7]:
cal_homo_trans_mat_0_to_n(6)

array([[-0.5732233 , -0.73919892, -0.35355339, -0.70710678],
       [ 0.73919892, -0.28033009, -0.61237244, -1.22474487],
       [ 0.35355339, -0.61237244,  0.70710678,  3.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [8]:
cal_homo_trans_mat_0_to_n(3)

array([[-0.35355339,  0.8660254 , -0.35355339,  0.        ],
       [-0.61237244, -0.5       , -0.61237244,  0.        ],
       [-0.70710678,  0.        ,  0.70710678,  2.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
cal_homo_trans_mat_0_to_n(3)[:3, :3]

array([[-0.35355339,  0.8660254 , -0.35355339],
       [-0.61237244, -0.5       , -0.61237244],
       [-0.70710678,  0.        ,  0.70710678]])

<table>
  <tr>
    <td><font size="5" color="yellow">
Inverse Kinematics: Position IK
</font></td>
  </tr>
  <tr>
    <td>
        <img src="images/kinematic_decoupling.png" width = "600" height = "300"/> 
    </td>
  </tr>
<tr>
    <td>
        <img src="images/ik/1.png" width = "200" height = "150"/> 
        <img src="images/ik/2.png" width = "200" height = "150"/> </td>
  </tr>
</table>

<table>
    <tr>
        <td><img src="images/vecC.png" width = "600" height = "300"/></td> 
    </tr>
    <tr>
        <td>
            <img src="images/ik/3.png" width = "600" height = "300"/> 
        </td>
  </tr>
</table>

<table>
  <tr>
    <td>
       <td><img src="images/vecA_B.png" width = "300" height = "100"/></td> 
    </td>
  </tr>
<tr>
    <td><img src="images/ik/4.png" width = "600" height = "300"/> </td>
    <td><img src="images/ik/5.png" width = "600" height = "300"/> </td>
    <td><img src="images/ik/6.png" width = "600" height = "300"/> </td>
  </tr>
 </table>

<table>
  <tr>
    <td><img src="images/ik/7.png" width = "600" height = "300"/> </td>
  </tr>
 </table>

<table>
  <tr>
    <td><font size="5" color="yellow">
Inverse Kinematics: Orientation IK
</font></td>
  </tr>
<tr>
    <td>
        <img src="images/ik/8.png" width = "500" height = "250"/> 
  </tr>
<tr>
    <td>
        <img src="images/ik/9.png" width = "500" height = "250"/> 
  </tr>
<tr>
    <td>
        <img src="images/ik/10.png" width = "500" height = "250"/> 
  </tr>
<tr>
    <td>
        <img src="images/ik/11.png" width = "300" height = "150"/> 
  </tr>
</table>

In [10]:
def inv_kinematics(eef_pose):
    '''Compute joint velocities based on the robot's end-effector pose (x, y, z, phi, theta, psi)
    '''
    x, y, z, phi, theta, psi = eef_pose # EULER ANGLES (Z(phi)-Y(theta)-Z(psi))

    print("End-Effector Pose: [" + str(x) + " " + str(y) + " " + str(z) + " " + str(phi) + " " + str(theta) + " " + str(psi) + "]")

    q = np.zeros(6)

    # p_06 = p_04 + d6*R_06@[0 0 1]^T
    p_06 = np.array([x, y, z])
    
    d6 = params[2, :][5]
    R_06 = np.array([[np.cos(phi), -np.sin(phi), 0], [np.sin(phi), np.cos(phi), 0], [0, 0, 1]]) \
            @ np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]]) \
            @ np.array([[np.cos(psi), -np.sin(psi), 0], [np.sin(psi), np.cos(psi), 0], [0, 0, 1]]) # EULER ANGLES (Z(phi)-Y(theta)-Z(psi))
    p_04 = p_06 - d6 * R_06 @ np.array([0, 0, 1])

    x_04, y_04, z_04 = p_04

    # Joint 1 angle 
    q[0] = np.arctan2(y_04, x_04) # it can also be (np.arctan2(y_04, x_04) + pi)
    
    # print((b*b + c - a2*a2 - d4*d4) / (2*a2*d4))
    cos_q3 = ((z_04 - params[2,:][0])**2 + x_04**2 + y_04**2 - params[0,:][1]**2 - params[2,:][3]**2) \
            / (2 * params[0,:][1] * params[2,:][3])
    
    # Joint 3 angle
    if np.isclose(cos_q3, 1):
        q[2] = 0
        q[1] = np.arctan2(z_04 - params[2,:][0], np.sqrt(x_04**2 + y_04**2))
    elif np.isclose(cos_q3, -1):
        q[2] = np.pi
    elif abs(cos_q3) < 1:
        q[2] = np.arctan2(np.sqrt(1 - cos_q3**2), cos_q3)

    alpha = np.arctan2((z_04 - params[2,:][0]), np.sqrt(x_04**2 + y_04**2)) # angle α formed by a and c
    beta = np.arctan2(params[2,:][3]*np.sin(q[2]), (params[0,:][1] + params[2,:][3]*np.cos(q[2])))  # angle β formed by a and a2
    # Joint 2 angle
    q[1] = alpha - beta


    T_01 = homo_trans_mat(params[:, 0][0], params[:, 0][1], params[:, 0][2], q[0])
    T_12 = homo_trans_mat(params[:, 1][0], params[:, 1][1], params[:, 1][2], q[1])
    T_23 = homo_trans_mat(params[:, 2][0], params[:, 2][1], params[:, 2][2], q[2] + np.pi/2)

    T_02 = T_01 @ T_12
    T_03 = T_02 @ T_23

    R_03 = T_03[:3, :3]
    R_36 = R_03.T @ R_06

    if (abs(R_36[2][2]) < 1):
        sin_thetaQ = np.sqrt(1 - R_36[2][2]**2)
    
        phi = np.arctan2(R_36[1][2], R_36[0][2])
        theta = np.arctan2(sin_thetaQ, R_36[2][2])
        psi = np.arctan2(R_36[2][1], -R_36[2][0])
    elif R_36[2][2] == 1:
        phi = 0
        theta = 0
        psi = np.arctan2(R_36[1][0], R_36[0][0])
    elif R_36[2][2] == -1:
        phi = np.arctan2(-R_36[0][1], -R_36[0][0])
        theta = np.pi
        psi = 0

    # Joint angle 4
    q[3] = phi
    # Joint angle 5
    q[4] = theta
    # Joint angle 6
    q[5] = psi

    return q   

In [11]:
eef_pose_1 = [2.422663680622242,0.05544267656769389,2.4863534126490863,-0.22263396888436274,0.6296077413830932, -1.9223913750523003]

In [12]:
inv_kinematics(eef_pose_1)

End-Effector Pose: [2.422663680622242 0.05544267656769389 2.4863534126490863 -0.22263396888436274 0.6296077413830932 -1.9223913750523003]


array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])

In [13]:
eef_pose = [2.42266368, 0.05544268, 2.48635341, -0.22263397, 0.62960774, -1.92239138]
joint_angles = inv_kinematics(eef_pose)
print(joint_angles)

End-Effector Pose: [2.42266368 0.05544268 2.48635341 -0.22263397 0.62960774 -1.92239138]
[0.1        0.2        0.3        0.4        0.5        0.59999999]


In [60]:
# def homo_trans_mat(a, alpha, d, q):
#     c_q = 0 if (q == np.pi/2 or q == -np.pi/2) else np.cos(q)
#     alpha_q = 0 if (alpha == np.pi/2 or alpha == -np.pi/2) else np.cos(alpha)

#     T1 = np.array([
#         [c_q, -np.sin(q), 0, 0],
#         [np.sin(q), c_q, 0, 0],
#         [0, 0, 1, 0],
#         [0, 0, 0, 1]
#     ])

#     T2 = np.block([
#         [np.eye(3), np.array([[0], [0], [d]])],
#         [np.zeros((1, 3)), 1]
#     ])

#     T3 = np.block([
#         [np.eye(3), np.array([[a], [0], [0]])],
#         [np.zeros((1, 3)), 1]
#     ])

#     T4 = np.array([
#         [1, 0, 0, 0],
#         [0, alpha_q, -np.sin(alpha), 0],
#         [0, np.sin(alpha), alpha_q, 0],
#         [0, 0, 0, 1]
#     ])

#     T = T1 @ T2 @ T3 @ T4

#     return T 

In [17]:
def ht(q, d, a, alpha):
    c_q = 0 if (q == np.pi/2 or q == -np.pi/2) else np.cos(q)
    alpha_q = 0 if (alpha == np.pi/2 or alpha == -np.pi/2) else np.cos(alpha)

    T1 = np.array([
        [c_q, -np.sin(q), 0, 0],
        [np.sin(q), c_q, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T2 = np.block([
        [np.eye(3), np.array([[0], [0], [d]])],
        [np.zeros((1, 3)), 1]
    ])

    T3 = np.block([
        [np.eye(3), np.array([[a], [0], [0]])],
        [np.zeros((1, 3)), 1]
    ])

    T4 = np.array([
        [1, 0, 0, 0],
        [0, alpha_q, -np.sin(alpha), 0],
        [0, np.sin(alpha), alpha_q, 0],
        [0, 0, 0, 1]
    ])

    T = T1 @ T2 @ T3 @ T4

    return T   

In [46]:
def ik(xi):
    x, y, z, phi, theta, psi = xi
    
    a = np.array([0, 1, 0, 0, 0, 0])
    d = np.array([1, 0, 0, 1, 0, 1])
    alpha = np.array([np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0])

    p06 = np.array([x, y, z])

    R06 = (
        np.array([
            [np.cos(phi), -np.sin(phi), 0], 
            [np.sin(phi), np.cos(phi), 0], 
            [0, 0, 1]
        ]) @ np.array([
            [np.cos(theta), 0, np.sin(theta)], 
            [0, 1, 0], 
            [-np.sin(theta), 0, np.cos(theta)]
        ]) @ np.array([
            [np.cos(psi), -np.sin(psi), 0], 
            [np.sin(psi), np.cos(psi), 0], 
            [0, 0, 1]])
    )

    p04 = p06 - d[5] * R06 @ np.array([0, 0, 1])

    xc, yc, zc = p04

    q = [0] * 6
    q[0] = np.arctan2(yc, xc)

    cosq3 = ((zc - d[0])**2 + xc**2 + yc**2 - a[1]**2 - d[3]**2) / (2 * a[1] * d[3])

    if np.isclose(cosq3, 1):
        q[2] = 0
        q[1] = np.arctan2(zc - d[0], np.sqrt(xc**2 + yc**2))
    elif np.isclose(cosq3, -1):
        q[2] = np.pi
    elif abs(cosq3) < 1:
        q[2] = np.arctan2(np.sqrt(1 - cosq3**2), cosq3)

    q[1] = (np.arctan2(zc - d[0], np.sqrt(xc**2 + yc**2)) - np.arctan2(d[3] * np.sin(q[2]), a[1] + d[3] * np.cos(q[2])))

    T01 = ht(q[0], d[0], a[0], alpha[0])
    T12 = ht(q[1], d[1], a[1], alpha[1])
    T23 = ht(q[2] + np.pi/2, d[2], a[2], alpha[2])

    print(str(q[0])+' '+str(d[0])+' '+str(a[0])+' '+str(alpha[0]))
    print(T01)
    print(T12)
    print(T23)

    T02 = T01 @ T12
    T03 = T02 @ T23

    R03 = T03[:3, :3]
    R36 = R03.T @ R06

    R = R36

    # print(R03)
    # print(R36)

    if abs(R[2, 2]) < 1: 
        phi = np.arctan2(R[1, 2], R[0, 2])
        theta = np.arctan2(np.sqrt(1 - R[2, 2]**2), R[2, 2])
        psi = np.arctan2(R[2, 1], -R[2, 0])
    elif R[2, 2] == 1:
        phi = 0
        theta = 0
        psi = np.arctan2(R[1, 0], R[0, 0])
    elif R[2, 2] == -1:
        phi = np.arctan2(-R[0, 1], -R[0, 0])
        theta = np.pi
        psi = 0

    # Joint angle 4
    q[3] = phi
    # Joint angle 5
    q[4] = theta
    # Joint angle 6
    q[5] = psi

    return np.array(q) 

In [47]:
xi = [2.42266368, 0.05544268, 2.48635341, -0.22263397, 0.62960774, -1.92239138]
ans = ik(xi)
print(ans)

0.1000000020164498 1 0 1.5707963267948966
[[ 0.99500417  0.          0.09983342  0.        ]
 [ 0.09983342  0.         -0.99500417  0.        ]
 [ 0.          1.          0.          1.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.98006658 -0.19866933  0.          0.98006658]
 [ 0.19866933  0.98006658  0.          0.19866933]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
[[-0.29552021  0.          0.95533649  0.        ]
 [ 0.95533649  0.          0.29552021  0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.          1.        ]]
[0.1        0.2        0.3        0.4        0.5        0.59999999]


In [ ]:
[[-0.35355339  0.8660254  -0.35355339]
 [-0.61237244 -0.5        -0.61237244]
 [-0.70710678  0.          0.70710678]]
[[-5.00000000e-01  8.66025404e-01  1.01465364e-17]
 [-8.66025404e-01 -5.00000000e-01  1.24758465e-17]
 [-9.31020989e-17 -1.63515442e-17  1.00000000e+00]]